In [169]:
import os
import json
from openai import OpenAI
from tqdm import tqdm
import json
import os
import re
import dotenv


In [170]:
import csv
import pandas as pd

# 创建 CSV 读取器
file = open('data/result_mis_letter_zero.csv', 'r', newline='', encoding='utf-8')

# 读取 CSV 文件
data = pd.read_csv(file)

In [171]:
# system = "You are GPT-4, answer my questions as if you were an expert in the field."


In [203]:
client = OpenAI(
    # This is the default and can be omitted
    api_key= os.getenv('OPENAI_API_KEY'),
    base_url='https://api.aiproxy.io/v1'
)

def generate_text(inputs):
    messages=[
          {"role": "user", "content": f'{inputs}'}
              ]
    # base_prompt = prompts['important']
    response = client.chat.completions.create(
            # model="text-davinci-003",
            model="gpt-4-turbo",
            # model="gpt-3.5-turbo",
            messages=messages,
            # stream=True
            temperature=0.6,
            max_tokens=512,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
            )
    return response.choices[0].message.content

In [193]:
# from openai import OpenAI
# client = OpenAI()

# def get_embedding(text, model="text-embedding-3-large"):
#    text = text.replace("\n", " ")
#    return client.embeddings.create(input = [text], model=model).data[0].embedding
# ori = 'hello'
# perb = 'how are u'
# emb = get_embedding(ori)

In [194]:
generate_text('hello')

'Hello! How can I assist you today?'

In [75]:
ramdom_data = data.sample(10, random_state=1)

In [177]:
ramdom_data = data

In [195]:
from tqdm import tqdm
for index, row in tqdm(ramdom_data.iterrows()):
    # print(f"q is {row['Content']}, the answer is {row['label']}")
    q = 'Q: Take the {index} letters of the words in {question} and concatenate them.'
    target,question,model_output=row['label'],q.format(index=row['ordinal_index'], question=row['Content']),row['pred']

    check_prompt = ('Please determine what type of error the prediction is based on the label. \n'
        'The question is: {question}, \nand the prediction is: {model_output}, \nthe label is: {target}. \n'
        'The types of the answer are [Totally correct, Method is wrong, Method correct but process wrong, Prediction correct but process wrong]\n'
        'Please always give the choice first, and explain it.\n'
        'For examples: Totally correct. Because...; Method is wrong. Because...; Method correct but process wrong. Because...; Lazy thinking. Because...; Result correct but process wrong. Because...\n'
        'The choice of "Method is wrong" means that the steps to solve the problem are wrong, e.g. the original step has 5 steps but the output process has only 3 steps, or an incorrect understanding of the question, etc.\n'.format(question=question,target=target,model_output=model_output)
        ).strip()
    
    output = generate_text(check_prompt)
    # 使用字符串的 split 方法将字符串分割成两部分
    label, explanation = output.split(".", 1)
    # label = label.split('is')[-1]
    ramdom_data.at[index, 'class'] = label.replace('\n', '\\n').strip()
    ramdom_data.at[index, 'explain'] = explanation.replace('\n', '\\n').strip()


200it [05:49,  1.75s/it]


In [196]:
ramdom_data

,label,num,ordinal_index,Content,pred,class,explain
0,MC,0,first,Morgan Calderon,"The first letter of each word in ""Morgan Calde...",Method is wrong,Because the prediction incorrectly concatenate...
1,rr,2,third,Mervin Carroll,The third letter of each word in Mervin Carrol...,Method correct but process wrong,Because the prediction correctly identified th...
2,ec,1,second,Bert Schwartz,The second letters of the words in Bert Schwar...,Method is wrong,Because the prediction incorrectly assumes tha...
3,JKPR,0,first,Jonathon Kelly Parra Rosas,J K P R R are the first letters of each word i...,Prediction correct but process wrong,Because the prediction correctly identified th...
4,raln,2,third,Darren Diaz Ellis Conner,"To find the answer, follow these steps:\n\n1. ...",Prediction correct but process wrong,Because the prediction provided the correct an...
...,...,...,...,...,...,...,...
195,ezal,2,third,Drew Vazquez Chase Keller,The third letter of each word in Drew Vazquez ...,Prediction correct but process wrong,Because the prediction correctly identifies th...
196,tiol,3,fourth,Weston Hamilton Lugo Arellano,"To find the answer, follow these steps:\n\n1. ...",Method correct but process wrong,Because the prediction correctly identifies th...
197,uh,3,fourth,August Archer,"The fourth letter of the word ""August"" is ""g""....",Prediction correct but process wrong,Because the prediction correctly identifies th...
198,aa,1,second,Marlin Garza,"The second letters of the words ""Marlin"" and ""...",Method is wrong,Because the prediction incorrectly took the se...


In [197]:

columns = list(ramdom_data.columns)
columns.index('class')


5

In [198]:
columns.insert(columns.index('Content'), columns.pop(columns.index('class')))  # 将第一列移动到第三列位置
columns.insert(columns.index('Content'), columns.pop(columns.index('explain')))
label_ori = ramdom_data.reindex(columns=columns)


In [199]:
label_ori.to_csv('35result_class_mis_letter_zero.csv', index=False, encoding='utf-8')

# GSM8K

In [202]:
import json
content = []
with open('data/GSM8K_mis_zero.jsonl', 'r')as file:
    for line in file:
        data = json.loads(line)
        content.append(data)

In [204]:
from tqdm import tqdm
for id, item in tqdm(enumerate(content)):
    # print(f"q is {row['Content']}, the answer is {row['label']}")
    question = item['question']
    target,question,model_output=item['ground truth'],item['question'],item['pred']

    check_prompt = ('Please determine what type of error the prediction is based on the label. \n'
        'The question is: {question}, \nand the prediction is: {model_output}, \nthe label is: {target}. \n'
        'The types of the answer are [Totally correct, Method is wrong, Method correct but process wrong, Prediction correct but process wrong]\n'
        'Please always give the choice first, and explain it.\n'
        'For examples: Totally correct. Because...; Method is wrong. Because...; Method correct but process wrong. Because...; Lazy thinking. Because...; Result correct but process wrong. Because...\n'
        'The choice of "Method is wrong" means that the steps to solve the problem are wrong, e.g. the original step has 5 steps but the output process has only 3 steps, or an incorrect understanding of the question, etc.\n'.format(question=question,target=target,model_output=model_output)
        ).strip()
    
    # print(check_prompt)
    # print('--seg--')
    output = generate_text(check_prompt)
    # 使用字符串的 split 方法将字符串分割成两部分
    label, explain = output.split(".", 1)
    item['verify'] = {'label': label, 'explain': explain}



0it [00:00, ?it/s]

100it [08:32,  5.12s/it]


In [205]:
output_file = 'GSM8K_mis_zero_val.jsonl'
with open(output_file, 'w') as f:
    # 遍历 JSON 数据列表
    for json_data in content:
        # 将每个 JSON 对象转换为字符串，并添加换行符
        json_line = json.dumps(json_data) + '\n'
        # 将 JSON 对象写入文件
        f.write(json_line)

In [211]:
content[2]

{'question': 'Kim sleepwalks, to monitor her sleeping hours, she installs a camera in her room. Kim usually goes to sleep at 10 p.m. and wakes up at 6 a.m. She is a sleepwalker. One day, after reviewing the cameras, she finds that she woke up that day and was sleepwalking from 2:15 to 2:38 am.  Also, that day she woke up 5 minutes earlier than usual to go to the bathroom. How many minutes did she sleep on her bed that day?',
 'pred': 'To determine how many minutes Kim slept on her bed that day, we first need to find out how long she slept from the time she went to bed until she woke up and sleepwalked. We know that she usually goes to bed at 10 p.m. and wakes up at 6 a.m., so she typically sleeps for 8 hours and 15 minutes (480 minutes).\n\nHowever, on the day in question, she woke up earlier than usual to go to the bathroom, so she only slept for the full 8 hours. Therefore, she slept for 480 minutes - the 5 minutes she was up in the bathroom = 475 minutes on her bed that day before t

In [6]:
import json
content = []
with open('GSM8K_mis_zero_val.jsonl', 'r')as file:
    for line in file:
        data = json.loads(line)
        content.append(data)

In [9]:
label_counts = {}

for item in content:
    label = item['verify']['label']
    if label in label_counts:
        label_counts[label] += 1
    else:
        label_counts[label] = 1

print(label_counts)

{'Method correct but process wrong': 23, 'Totally correct': 43, 'Method is wrong': 32, 'Prediction correct but process wrong': 2}
